<a href="https://colab.research.google.com/github/jhphan/ML-Notebooks/blob/main/tcga-scikit-learn-logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# authenticate to gcloud
!gcloud auth application-default login

In [9]:
# udpate these variables
client_project = 'cgc-05-0051'
cancer_type = 'TCGA-GBM'
project = 'isb-cgc-bq'

In [3]:
# Load dependencies
from google.cloud import bigquery
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [4]:
# Create a client to access the data within BigQuery
client = bigquery.Client(client_project)

/usr/local/lib/python3.7/dist-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [10]:
# Get the list of unique proteins to create a pivot table
demo_gender = client.query(
  ("""
    SELECT count(demo__gender) as number, demo__gender from `{}.TCGA.clinical_gdc_current` where proj__project_id = '{}' group by demo__gender
  """).format(project, cancer_type)
).result().to_dataframe()
demo_gender

,number,demo__gender
0,0,None
1,230,female
2,366,male
3,3,not reported


In [ ]:
# Get the list of unique proteins to create a pivot table
protein_names = client.query(
  ("""
    SELECT CONCAT(
      '("',
      STRING_AGG(
        DISTINCT CONCAT('p_', REPLACE(protein_name, "-", "_")), '", "'
      ),
      '")'
    ) AS protein_names 
    FROM `{}.TCGA.protein_expression_hg38_gdc_current`
  """).format(project)
).result().to_dataframe()['protein_names'][0]
protein_names

In [ ]:
# Join clinical and protein expression data to create a labeled pivot table
protein_expression = client.query(
  ("""
    SELECT * FROM (
      SELECT 
        pe.case_barcode AS sample,
        labels.survival_label AS label,
        CONCAT('p_', REPLACE(pe.protein_name, "-", "_")) AS protein_name,
        pe.protein_expression AS protein_expression
      FROM `isb-cgc-bq.TCGA.protein_expression_hg38_gdc_current` AS pe
      INNER JOIN (
        SELECT *
        FROM (
          SELECT
            submitter_id,
            demo__vital_status,
            demo__days_to_death,
            diag__days_to_last_follow_up,
            CASE
              WHEN demo__vital_status = 'Dead' AND demo__days_to_death < 365*5 THEN 0
              WHEN (
                (demo__vital_status = 'Dead' AND demo__days_to_death >= 365*5)
                OR (demo__vital_status = 'Alive' AND diag__days_to_last_follow_up >= 365*5)
              ) THEN 1
            END AS survival_label
          FROM `{}.TCGA.clinical_gdc_current`
          WHERE proj__project_id = '{}'
        )
        WHERE survival_label IS NOT NULL
      ) labels
      ON labels.submitter_id = pe.case_barcode
    )
    PIVOT (
      MAX(protein_expression) FOR protein_name IN {}
    )
    ORDER BY sample ASC
  """).format(project, cancer_type, protein_names)
).result().to_dataframe()
protein_expression

In [ ]:
# find and remove samples that have no valid protein values (all missing)
samples_notnull = protein_expression.loc[:, 'p_AR':].notnull().sum(axis=1)>0
# find and remove proteins that have any NULL values
proteins_notnull = pd.Series(
  [True, True],
  index=['sample', 'label']
).append(
  protein_expression.loc[:, 'p_AR':].isnull().sum(axis=0)==0
)
# create a new filtered DF
protein_expression_filtered = protein_expression.loc[samples_notnull, proteins_notnull]
protein_expression_filtered

In [ ]:
# remove sample names from table
protein_expression_filtered.pop('sample')

# split data into train, val, and test sets
train_data = protein_expression_filtered.sample(frac=0.8, random_state=1)
val_data = protein_expression_filtered.drop(train_data.index)
test_data = val_data.sample(frac=0.5, random_state=1)
val_data = val_data.drop(test_data.index)

data = dict()
data['train_y'] = train_data.pop('label')
data['val_y'] = val_data.pop('label')
data['test_y'] = test_data.pop('label')

# calculate fold change and get top 20 proteins
train_fold_change = abs(train_data.loc[data['train_y']==1, 'p_AR':].mean(axis=0) - train_data.loc[data['train_y']==0, 'p_AR':].mean(axis=0))
top_train_fold_change = train_fold_change.sort_values(ascending=False) #.head(10)

scaler = StandardScaler()
data['train_x'] = scaler.fit_transform(train_data.loc[:,top_train_fold_change.index])
data['val_x'] = scaler.transform(val_data.loc[:,top_train_fold_change.index])
data['test_x'] = scaler.transform(test_data.loc[:,top_train_fold_change.index])

data['scaler'] = scaler

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score

clf = svm.LinearSVC(max_iter=2000)
clf.fit(data['train_x'], data['train_y'])
pred = clf.predict(data['val_x'])
acc = accuracy_score(data['val_y'], pred)
acc, clf


In [ ]:
data['val_y'], pred

In [ ]:
# build DNN model

from keras.layers import Input, Dense, Dropout
from keras.models import Model

input_features = data['train_x'].shape[1]

# build the network
inputs = Input(shape=(input_features,), name='input')
x = Dense(64, activation='relu', name='hidden1', kernel_regularizer='l2')(inputs)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu', name='hidden2', kernel_regularizer='l2')(x)
x = Dense(16, activation='relu', name='hidden3')(x)
x = Dense(8, activation='relu', name='hidden4')(x)
x = Dense(4, activation='relu', name='hidden5')(x)
prediction = Dense(1, activation='sigmoid', name='final')(inputs)
model = Model(inputs=inputs, outputs=prediction)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
model.fit(x=data['train_x'], y=data['train_y'], \
          batch_size=32, epochs=300, verbose=1, validation_data=(data['val_x'], data['val_y']))